<a href="https://colab.research.google.com/github/Ayanlola2002/DATA-SCIENCE-PROJECTS/blob/master/GlobalStores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mounting Gdrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from geopy.distance import vincenty
import time
#pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points

In [3]:
df=pd.read_excel('/content/drive/MyDrive/CIC/global-superstore.xlsx')


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Row ID          51290 non-null  int64         
 1   Order ID        51290 non-null  object        
 2   Order Date      51290 non-null  datetime64[ns]
 3   Ship Date       51290 non-null  datetime64[ns]
 4   Ship Mode       51290 non-null  object        
 5   Customer ID     51290 non-null  object        
 6   Customer Name   51290 non-null  object        
 7   Segment         51290 non-null  object        
 8   City            51290 non-null  object        
 9   State           51290 non-null  object        
 10  Country         51290 non-null  object        
 11  Postal Code     9994 non-null   float64       
 12  Market          51290 non-null  object        
 13  Region          51290 non-null  object        
 14  Product ID      51290 non-null  object        
 15  Ca

In [5]:
#df['City'].isnull().sum()
df['City'].value_counts()

New York City           915
Los Angeles             747
Philadelphia            537
San Francisco           510
Santo Domingo           443
                       ... 
Barletta                  1
Río Bravo                 1
Le Pré-Saint-Gervais      1
Passau                    1
Guelmim                   1
Name: City, Length: 3636, dtype: int64

In [6]:
df['address']=df['City']+','+df['Country']

In [7]:
df['address'].nunique()


3703

In [8]:
location = [x for x in df['address'].unique().tolist() if type(x) == str]
latitude = []
longitude =  []
for i in range(0, len(location)):
  # remove things that does not seem usefull here
  try:
    address = location[i] 
    geolocator = Nominatim(user_agent="ny_explorer")
    loc = geolocator.geocode(address)
    latitude.append(loc.latitude)
    longitude.append(loc.longitude)
    print('The geographical coordinate of location {} are {}, {}.'.format(i,loc.latitude, loc.longitude))
  except:
    # in the case the geolocator does not work, then add nan element to list
    # to keep the right size
    latitude.append(np.nan)
    longitude.append(np.nan)


The geographical coordinate of location 0 are 40.7127281, -74.0060152.
The geographical coordinate of location 1 are -34.4243941, 150.89385.
The geographical coordinate of location 2 are -27.4689682, 153.0234991.
The geographical coordinate of location 3 are 52.5015217, 13.4025498.
The geographical coordinate of location 4 are 14.693425, -17.447938.
The geographical coordinate of location 5 are -33.8548157, 151.2164539.
The geographical coordinate of location 6 are -41.1354918, 174.8396887.
The geographical coordinate of location 7 are -37.7878809, 175.281788.
The geographical coordinate of location 8 are 38.5815719, -121.4943996.
The geographical coordinate of location 9 are 43.207178, -71.537476.
The geographical coordinate of location 10 are 38.8051095, -77.0470229.
The geographical coordinate of location 11 are 34.5260109, 69.1776838.
The geographical coordinate of location 12 are 17.333333, 42.666667.
The geographical coordinate of location 13 are -22.7433969, -46.3735044.
The geo

In [9]:
# create a dataframe with the locatio, latitude and longitude
df_ = pd.DataFrame({'address':location, 
                    'location_latitude': latitude,
                    'location_longitude':longitude})

In [10]:
# merge on Restaurant_Location with rest_df to get the column 
new_df =df.merge(df_, on='address', how='left')

In [15]:
new_df.drop('address',axis=1,inplace=True)

In [16]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51290 entries, 0 to 51289
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Row ID              51290 non-null  int64         
 1   Order ID            51290 non-null  object        
 2   Order Date          51290 non-null  datetime64[ns]
 3   Ship Date           51290 non-null  datetime64[ns]
 4   Ship Mode           51290 non-null  object        
 5   Customer ID         51290 non-null  object        
 6   Customer Name       51290 non-null  object        
 7   Segment             51290 non-null  object        
 8   City                51290 non-null  object        
 9   State               51290 non-null  object        
 10  Country             51290 non-null  object        
 11  Postal Code         9994 non-null   float64       
 12  Market              51290 non-null  object        
 13  Region              51290 non-null  object    

In [18]:
new_df.to_csv('GlobalStores2.csv',index=False)

In [ ]:
df.head(4)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,Postal Code,Market,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority,lat,long
0,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,United States,10024.0,US,East,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical,NaN,NaN
1,26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,Australia,NaN,APAC,Oceania,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical,NaN,NaN
2,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,Australia,NaN,APAC,Oceania,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium,NaN,NaN
3,13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,Germany,NaN,EU,Central,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium,NaN,NaN
